# Welcome to my Capstone project !

In [1]:
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
import requests
from geopy.geocoders import Nominatim
import folium
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
url="https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M"
raw=requests.get(url).text
#Get Text Version of LXML code

soup = BeautifulSoup(raw,"lxml")
My_table = soup.find("table",{"class":"wikitable sortable"}) #obtain only table code and its contents from the Wikipedia Page
pcode=[]
borou=[]
neigh=[]

for row in My_table.findAll("tr"):
    cells = row.findAll("td")
    #For each "tr", assign each "td" to a variable.
    if len(cells)==3:
        pcode.append(cells[0].find(text=True))
        borou.append(cells[1].find(text=True))
        neigh.append(cells[2].find(text=True))
        
column=['Postcode','Borough','Neighborhood']
mainframe=pd.DataFrame(columns=column) #Generate empty dataframe

for i in range(0,len(pcode)): #Add all data into dataframe, except 'Not assigned' Boroughs
    if borou[i]!="Not assigned":
        mainframe.loc[i]=[pcode[i],borou[i],neigh[i]]

mainframe.reset_index(drop=True)
pcodes=mainframe['Postcode'].unique()     #Get list of postal codes

In [3]:
for i in pcodes:
    indexes=mainframe.loc[mainframe['Postcode']==i].index #Get list of indexes corresponding to each postcode
    text=mainframe['Neighborhood'][indexes[0]].replace('\n','')
    for j in range(0,len(indexes)):
        if j !=0:
            text_add=mainframe['Neighborhood'][indexes[j]].replace('\n','')
            text=text+", "+text_add #Prepare new text data for each unique Postcode
            mainframe.drop(indexes[j],inplace=True) #Drop rows except the merged row
    mainframe['Neighborhood'][indexes[0]]=text

In [4]:
mainframe.reset_index(drop=True,inplace=True)

In [5]:
NAindexes=mainframe.loc[mainframe['Neighborhood']=='Not assigned'].index #Get list of indexes corresponding to 'Not assigned' Neighborhood
for a in NAindexes:
    mainframe['Neighborhood'][a]=mainframe['Borough'][a]

In [6]:
#print("Number of rows in filtered dataframe: {}".format(mainframe.shape[0]))
#mainframe.head(11)

In [7]:
#Initialise new columns in the mainframe
mainframe['Latitude']=None 
mainframe['Longitude']=None
#Read CSV file
toadd=pd.DataFrame(pd.read_csv('http://cocl.us/Geospatial_data'))

In [8]:
#Correlate and add latitudes and longitudes from 'toadd' into mainframe
for i in pcodes:
    for j in toadd['Postal Code']:
        if i ==j:
            mainframe['Latitude'][mainframe.loc[mainframe['Postcode']==i].index]=toadd['Latitude'][toadd.loc[toadd['Postal Code']==j].index]
            mainframe['Longitude'][mainframe.loc[mainframe['Postcode']==i].index]=toadd['Longitude'][toadd.loc[toadd['Postal Code']==j].index]

In [9]:
#mainframe.head(11)

In [10]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [11]:
# create map of Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(mainframe['Latitude'], mainframe['Longitude'], mainframe['Borough'], mainframe['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
#map_toronto

In [12]:
CLIENT_ID = 'WGMMNGSOYDZEJQIEMQGBAQFOBHULTPON0KLP1WF3H2Z3FZGO' # your Foursquare ID
CLIENT_SECRET = 'G2WZIRARGY5FO2O4XRPXFFYZ5SS5GLKR4HV5HUAPRKKVJAW5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WGMMNGSOYDZEJQIEMQGBAQFOBHULTPON0KLP1WF3H2Z3FZGO
CLIENT_SECRET:G2WZIRARGY5FO2O4XRPXFFYZ5SS5GLKR4HV5HUAPRKKVJAW5


In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
toronto_venues = getNearbyVenues(names=mainframe['Neighborhood'],
                                   latitudes=mainframe['Latitude'],
                                   longitudes=mainframe['Longitude']
                                  )

KeyError: 'groups'

In [ ]:
#toronto_venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe, at first column
toronto_onehot.drop('Neighborhood',axis=1,inplace=True)
toronto_onehot.insert(loc=0,column='Neighborhood',value=toronto_venues['Neighborhood'])

In [ ]:
#toronto_onehot.shape

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
#toronto_grouped

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    #print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
   # print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

#neighborhoods_venues_sorted.head()

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = mainframe

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.dropna(inplace=True)
toronto_merged.head() # check the last columns!

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters